In [ ]:
using Plots; pyplot()
using LaTeXStrings
using BasicBVP1D
using LinearAlgebra
using Dierckx

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfont=font(12), lw=2,ms=8)

# Test Problem for Convergence
$$
-u'' = \pi^2 \sin(\pi x), \quad 0<x<1, \quad u(0) = u(1) =0
$$
which has the exact solution, $u = \sin(\pi x)$.

Let us pretend we do not know the exact solution, but believe our code is correct, and want to check convergence.  We do the following:
1. First, we compute a solution on a very high resolution mesh, $n_{h}$
2. Then we compute the solution on a sequence of lower resolutions meshes, $n\ll n_{h}$, comparing the error between the low resolution solution with the high resolution one.
3. The trick is how to compare at common mesh points.  There are two options here:
   * One can simply use an interpolant on the high resolution method, and treat the interpolated solution as the exact solution.  
   * If the two meshes are conformal, such that the high resolution mesh includes the low resolution mesh's points, then you can make a directly comparison without interpolation, matching the values up.  This requires very careful checking of the indexes, to ensure that you have the right values.

We are, effectively, using this high resolution solution as a surrogate for the exact solution, which we would otherwise not know.

## Comparison with Interpolation

In [ ]:
f = x-> π^2 * sin(π * x);
a = 0;
b = 1;
n_vals = [5, 10, 20, 40, 80, 160, 320, 640] .-1;
Δx_vals = @. (b-a) / ( n_vals +1);

n_h = 5 * (2^10) - 1;
@show n_h;
Δx_h = (b-a)/(n_h + 1);
@show Δx_h;

In [ ]:
# construct the high res solution and its interpolant
problem_h = FiniteDifferenceBVPProblem(a, b, n_h, f);
assemble_system!(problem_h);
u_h = solve_bvp(problem_h);
u_itp = Spline1D([a; problem_h.x;b], [0; u_h; 0], k=1);

In [ ]:
err_vals = [];
for n in n_vals
    problem = FiniteDifferenceBVPProblem(a, b, n, f);
    assemble_system!(problem);
    u = solve_bvp(problem);
    err = norm(u .- u_itp.(problem.x), Inf);
    push!(err_vals, err);
end

In [ ]:
scatter(Δx_vals, err_vals, xscale=:log10, yscale=:log10, 
    label="Computed Error",legend=:bottomright)
plot!(Δx_vals, Δx_vals.^2, label=L"$\propto \Delta x^2$")
xlabel!(L"$\Delta x$")
title!("Error with Interpolated Surrogate Solution")

## Comparison with Mathced Meshes
Observe that with the values of `n` above, the meshes are conformal:

In [ ]:
n = n_vals[1];
problem = FiniteDifferenceBVPProblem(a, b, n, f);
@show problem.x;
@show problem_h.x;
@show problem.Δx/problem_h.Δx

Consequently, there are exactly `1024` fine mesh spacings between the coarse mesh points:

In [ ]:
@show problem.x|>collect;
@show problem_h.x[1024:1024:end]|>collect;

Conseqeuntly, we see that to compare the results at the same `x` values, we should space them out by a factor of `1024`.

Note that we use `|>collect` to see the concrete set of values instead of a `LinRange` type.

In [ ]:
err_vals = [];
for n in n_vals
    problem = FiniteDifferenceBVPProblem(a, b, n, f);
    assemble_system!(problem);
    u = solve_bvp(problem);
    n_coarse = Int(problem.Δx /problem_h.Δx); # cast to an integer
    err = norm(u .- u_h[n_coarse:n_coarse:end], Inf);
    push!(err_vals, err);
end

In [ ]:
scatter(Δx_vals, err_vals, xscale=:log10, yscale=:log10, 
    label="Computed Error",legend=:bottomright)
plot!(Δx_vals, Δx_vals.^2, label=L"$\propto \Delta x^2$")
xlabel!(L"$\Delta x$")
title!("Error with Matched Surrogate Solution")

## Comparison with Periodic Boundary Conditions
The same strategy can be used with other boundary conditions and other problems.  Consider the problem:
$$
-u'' = (2\pi)^2 \sin(2\pi x)
$$
with periodic boundary conditions.  

The one subtlety here is that you will want to use the `periodic=true` option in the interpolated approach, and you will also want to make sure to pass in the periodicity.  In the matched appraoch, you will also want to include the value at the left endpoint for comparison.

### Comparison with Interpolation

In [ ]:
f = x-> (2*π)^2 * sin(2*π * x);
a = 0;
b = 1;
n_vals = [5, 10, 20, 40, 80, 160, 320, 640] .-1;
Δx_vals = @. (b-a) / ( n_vals +1);

n_h = 5 * (2^10) - 1;
@show n_h;
Δx_h = (b-a)/(n_h + 1);
@show Δx_h;

In [ ]:
# construct the high res solution and its interpolant
problem_h = FiniteDifferenceBVPProblem(a, b, n_h, f, PeriodicBC(), PeriodicBC());
assemble_system!(problem_h);
u_h = solve_bvp(problem_h);
u_itp = Spline1D([problem_h.x;b], [u_h; u_h[1]], k=1, periodic=true);

In [ ]:
err_vals = [];
for n in n_vals
    problem = FiniteDifferenceBVPProblem(a, b, n, f, PeriodicBC(), PeriodicBC());
    assemble_system!(problem);
    u = solve_bvp(problem);
    err = norm((u .-u[1]) .- (u_itp.(problem.x) .- u_itp(a)), Inf);
    push!(err_vals, err);
end

Note that we have to shift both solutions (i.e. `u .- u[1]`) to correct for the fact that there is a kernel and both solutions are only known up to an additive constant.

In [ ]:
scatter(Δx_vals, err_vals, xscale=:log10, yscale=:log10, 
    label="Computed Error",legend=:bottomright)
plot!(Δx_vals, Δx_vals.^2, label=L"$\propto \Delta x^2$")
xlabel!(L"$\Delta x$")
title!("Error with Interpolated Surrogate Solution")

### Comparison with Matched Solution

In [ ]:
n = n_vals[1];
problem = FiniteDifferenceBVPProblem(a, b, n, f,PeriodicBC(), PeriodicBC());
@show problem.x;
@show problem_h.x;
@show problem.Δx/problem_h.Δx
@show problem.x|>collect;
@show problem_h.x[1:1024:end]|>collect;

In [ ]:
err_vals = [];
for n in n_vals
    problem = FiniteDifferenceBVPProblem(a, b, n, f,PeriodicBC(), PeriodicBC());
    assemble_system!(problem);
    u = solve_bvp(problem);
    n_coarse = Int(problem.Δx /problem_h.Δx); # cast to an integer
    err = norm( (u.-u[1]) .- (u_h[1:n_coarse:end] .-u_h[1]), Inf);
    push!(err_vals, err);
end

In [ ]:
scatter(Δx_vals, err_vals, xscale=:log10, yscale=:log10, 
    label="Computed Error",legend=:bottomright)
plot!(Δx_vals, Δx_vals.^2, label=L"$\propto \Delta x^2$")
xlabel!(L"$\Delta x$")
title!("Error with Matched Surrogate Solution")